In [ ]:
# !pip install dash pandas
# !pip install dash==2.0.0
#!pip install -q jupyter_dash==0.3.0
# !pip install -q dash-cytoscape
# !pip install typing-extensions
# !pip install pandas-datareader
# !pip install pmdarima

In [ ]:
import base64
import datetime
import io
from datetime import datetime
from jupyter_dash import JupyterDash
from dash import no_update
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
import dash
import plotly.express as px
import plotly.graph_objects as go
import plotly.tools as tls
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import to_datetime
from prophet import Prophet
from matplotlib import pyplot
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from datetime import datetime
from pandas import to_datetime
from prophet import Prophet
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose

/tmp/ipykernel_10279/1024821048.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_10279/1024821048.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/ipykernel_10279/1024821048.py:10: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [ ]:
import base64
import datetime
import io
import _strptime
from jupyter_dash import JupyterDash
from dash import no_update
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd


app = dash.Dash(external_stylesheets=[dbc.themes.DARKLY])

app.layout = html.Div(children=[
    html.Div([
                 html.H2("TIME SERIES ANALYSIS OF DATA", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading")
                ],className="row"),

        

        dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '98%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='Mytext'),
    html.Div(id='Mygraph0'),
    html.Div(id='Mygraph1')
    
])

def parser(s):
    return datetime.datetime.strptime(s, '%Y-%m-%d')


def model_predict(df):
  
  df.drop(df.columns.difference(['Date','Close']), 1, inplace=True)
  df.dropna(inplace = True)
  desc = df.describe()
  desc = desc.to_string()
  train_size = 0.80
  split_idx = round(len(df)* train_size)
  train = df.iloc[:split_idx]
  test = df.iloc[split_idx:]
  
  #ARIMA
  df = df.iloc[1: , :]
  model = sm.tsa.arima.ARIMA(train,order=(1,1,1))
  model_fit = model.fit()
  modelvalues = model_fit.predict(start=1, end=len(df))
  df['Predicted_ARIMA'] = modelvalues.values
  predicted_arima = df['Predicted_ARIMA']
  str_msg = ""
  str_msg = str_msg + model_fit.summary().as_text()

  from sklearn.metrics import r2_score 
  a =df['Close'] 
  b =df['Predicted_ARIMA']
  R_square = r2_score(a, b) 
  print('Coefficient of Determination for ARIMA Model', R_square)
  str_metric = 'Coefficient of Determination for ARIMA Model is : '
  str_metric = str_metric + str(R_square)

  #Auto ARIMA
  model = auto_arima(train, start_p=0, start_q=0, test='adf', max_p=7, 
                     max_q=7, m=1, d=0, seasonal=False, start_P=0, 
                     D=0, error_action='ignore', suppress_warnings=True, 
                     stepwise=True)
  forecast = model.predict(n_periods=len(df))
  forecast = pd.DataFrame(forecast,index = df.index,columns=['Prediction'])
  df['Prediction'] = forecast.values

  str_msg1 = ""
  str_msg1 = str_msg1 + model.summary().as_text()

  from sklearn.metrics import r2_score 
  a =df['Close'] 
  b =df['Prediction']
  R_square = r2_score(a, b) 
  print('Coefficient of Determination for AUTO ARIMA Model', R_square)
  str_metric1 = 'Coefficient of Determination for AUTO ARIMA Model is : '
  str_metric1 = str_metric1 + str(R_square)

  #SARIMA
  model_SARIMA=SARIMAX(train,order=(1,1,1),seasonal_order=(0,1,0,12))
  model_SARIMA_fit=model_SARIMA.fit()
  modelvalues1 = model_SARIMA_fit.predict(start=1,end=len(df))
  df['Predicted_SARIMA']=modelvalues1.values
  predicted_sarima = df['Predicted_SARIMA']
  residuals=df['Close']-df['Predicted_SARIMA']
  str_msg2 = ""
  str_msg2 = str_msg2 + model_SARIMA_fit.summary().as_text()

  from sklearn.metrics import r2_score 
  a =df['Close'] 
  b =df['Predicted_SARIMA']
  R_square = r2_score(a, b) 
  print('Coefficient of Determination for SARIMA Model', R_square)
  str_metric2 = 'Coefficient of Determination for SARIMA Model is : '
  str_metric2 = str_metric2 + str(R_square)
  df.reset_index(inplace = True)
  
  
  return(df, desc, str_msg, str_msg1, str_msg2, str_metric, str_metric1, str_metric2)


  #FB PROPHET
from sklearn.metrics import r2_score 
def fbmodel_predict(df1):  
  df1.reset_index(inplace = True)
  df1.drop(df1.columns.difference(['Date','Close']), 1, inplace=True)
  df1.columns = ['ds', 'y']
  df1['ds']= to_datetime(df1['ds'])

  train_size = 0.80
  split_idx = round(len(df1)* train_size)

  train1 = df1.iloc[:split_idx]
  test1 = df1.iloc[split_idx:]

  # define the model
  model = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
  # fit the model
  model.fit(df1)
  forecast_3 = model.predict(test1)
  df_past = pd.DataFrame(forecast_3)
  # define the period for which we want a prediction
  # future = list()
  # for i in range(3,13):
  #     date = '2022-%02d' % i
  #     future.append([date])
  # future = DataFrame(future)
  # future.columns = ['ds']
  # future['ds']= to_datetime(future['ds'])
  # # use the model to make a forecast
  # forecast_4 = model.predict(future)
  future = model.make_future_dataframe(periods=365)
  forecast_4 = model.predict(future)
  df_future = pd.DataFrame(forecast_4)
  # summarize the forecast
  #print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])
  a =test1['y'].values 
  b =df_past['yhat'].values
  R_square = r2_score(a, b)
  str_metric3 = 'Coefficient of Determination for FB Prophet Model is : '
  str_metric3 = str_metric3 + str(R_square)
  return (df_past, df_future, str_metric3)
  

def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')), parse_dates=[0], squeeze=True, date_parser=parser)
            # df,df1,df_past = model_predict(df)
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return df
    
@app.callback(Output('Mytext', 'children'),
              [Input('upload-data', 'filename'),
               Input('upload-data', 'contents')
              ]) 
def update_filename(filename, contents):
  children1 = html.Div()
  if contents:
    contents = contents[0]
    filename = filename[0]
    df = parse_contents(contents, filename)

    children1 = html.Div(children=[
                                   
                                   html.Hr(),
                                   html.H4("File Upload Successful", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading"),
                                   html.H5(filename, 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading1")
                           
    ]) 
    return children1
  else: 
    return dash.no_update



@app.callback(Output('Mygraph0', 'children'),
              [
              Input('upload-data', 'contents'),
              Input('upload-data', 'filename')                 
              ])
def update_graph0(contents, filename):
      children = html.Div()    
      if contents:
        contents = contents[0]
        filename = filename[0]
        df = parse_contents(contents, filename)
        desc = df.describe()
        str_msg0 = '\t\t\t\t\t\t\t\t This is the Summary of the RAW DATA SET \n\n\n'
        desc = str_msg0 + desc.to_string()
        #result = seasonal_decompose(df['Close'], model='multiplicative', period=365)  
        children = html.Div(children=[
                            
                  html.Hr(),
                  html.H4("RAW DATA from the uploaded file (stock prices)", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading0"),
                  

                  dcc.Graph(id='line-graph0', style={'display': 'inline-block', 'width':'59%', 'marginLeft':'15px'},
                  figure=go.Figure(data = [go.Scatter(
                                              x = df['Date'],
                                              y = df['Close'],
                                              mode = 'lines',
                                              name = 'Close',
                                              line={'color': 'orange'}),
                                           go.Scatter(
                                              x = df['Date'],
                                              y = df['Open'],
                                              mode = 'lines',
                                              name = 'Open',
                                              line={'color': 'green'}),
                                           go.Scatter(
                                              x = df['Date'],
                                              y = df['High'],
                                              mode = 'lines',
                                              name = 'High',
                                              line={'color': 'blue'}),
                                           go.Scatter(
                                              x = df['Date'],
                                              y = df['Low'],
                                              mode = 'lines',
                                              name = 'Low',
                                              line={'color': 'red'}),
                                           go.Scatter(
                                              x = df['Date'],
                                              y = df['Adj Close'],
                                              mode = 'lines',
                                              name = 'Adj Close',
                                              line={'color': 'brown'}),
                                          
                                   ],
                                    layout=go.Layout(title={'text':'Stock prices taken from RAW DATA',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     #plot_bgcolor='#070d30',
                                                     xaxis_title="Date", 
                                                     yaxis_title="Stock price in pounds (£)")
                                    ), className="Three columns"),

                  dcc.Textarea(id='run-model-echo0', readOnly='readonly', placeholder='',
                                value= desc,
                                rows='30',
                                style={'display': 'inline', 'width': '38%', 'height':'800', 'background-color': '#F5F5F5', 'marginLeft':'8px',
                                       'marginRight':'15px','opacity': 0.5, 'fontSize': 10, 'draggable': False, 'text-align':'left'}, 
                               className="three columns")

                  # dcc.Graph(id='line-graph01', style={'display': 'inline-block', 'width':'98%'},
                  # figure=go.Figure(data = [go.Scatter(
                  #                             x = df['Date'],
                  #                             y = result.trend,
                  #                             mode = 'lines',
                  #                             name = 'Trend',
                  #                             line={'color': 'orange'}),
                  #                          go.Scatter(
                  #                             x = df['Date'],
                  #                             y = result.seasonal,
                  #                             mode = 'lines',
                  #                             name = 'Seasonality',
                  #                             line={'color': 'green'}),
                  #                          go.Scatter(
                  #                             x = df['Date'],
                  #                             y = result.resid,
                  #                             mode = 'lines',
                  #                             name = 'Noise',
                  #                             line={'color': 'blue'}),
                  #                       ]))

                ])

        return children
      else: 
        return dash.no_update





@app.callback(Output('Mygraph1', 'children'),
              [
              Input('upload-data', 'contents'),
              Input('upload-data', 'filename')                 
              ])
def update_graph1(contents, filename):
      children = html.Div()    
      if contents:
        contents = contents[0]
        filename = filename[0]
        df = parse_contents(contents, filename)
        df = df.set_index(df.columns[0])
        df_pred, desc, str_msg, str_msg1, str_msg2, str_metric, str_metric1, str_metric2 = model_predict(df)
        df_past, df_future, str_metric3 = fbmodel_predict(df)
        # print()
        # print()
        # print(df_pred)
        # print()
        # print()
                
                     
        
        children = html.Div(children=[
                            
                  # html.Hr(),
                  # html.H4("Close prices of stocks for RAW DATA", 
                  #         style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                  #                'color':'white'}, 
                  #         className="text-center", id="heading0"),
                  

                  # dcc.Graph(id='line-graph0', style={'display': 'inline-block', 'width':'49%'},
                  # figure=go.Figure(data = [go.Scatter(
                  #                             x = df_pred['Date'],
                  #                             y = df_pred['Close'],
                  #                             mode = 'lines',
                  #                             name = 'Close',
                  #                             line={'color': 'orange'}
                  #                             ),
                  #                  ],
                  #                   layout=go.Layout(title={'text':'Close prices of stocks for RAW DATA',
                  #                                           'y':0.9,
                  #                                           'x':0.5,
                  #                                           'xanchor': 'center',
                  #                                           'yanchor': 'top'},
                  #                                    #plot_bgcolor='#070d30',
                  #                                    xaxis_title="Date", 
                  #                                    yaxis_title="Closing price in pounds (£)")
                  #                   ), className="Three columns"),

                  # dcc.Textarea(id='run-model-echo0', readOnly='readonly', placeholder='',
                  #               value= desc,
                  #               rows='30',
                  #               style={'display': 'inline', 'width': '28%', 'height':'1000', 'background-color': '#F5F5F5', 'marginLeft':'8px',
                  #                      'marginRight':'15px','opacity': 0.5, 'fontSize': 10, 'draggable': False, 'text-align':'center'}, 
                  #              className="three columns"),

                  html.Hr(),
                  html.H4("Outputs for ARIMA MODEL", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading1"),
                  

                  dcc.Graph(id='line-graph', style={'display': 'inline-block', 'width':'49%', 'marginLeft':'15px'},
                  figure=go.Figure(data = [go.Scatter(
                                              x = df_pred['Date'],
                                              y = df_pred['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}
                                              ),
                                   go.Scatter(
                                              x = df_pred['Date'],
                                              y = df_pred['Predicted_ARIMA'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),],
                                    layout=go.Layout(title={'text':'Close prices Prediction for ARIMA model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     #plot_bgcolor='#070d30',
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                                    ), className="Three columns"),
                            
                  
                  dcc.Textarea(id='run-model-echo', readOnly='readonly', placeholder='',
                                value= str_msg,
                                rows='30',
                                style={'display': 'inline', 'width': '48%', 'height':'1000', 'background-color': '#F5F5F5', 'marginLeft':'8px',
                                       'marginRight':'15px','opacity': 0.5, 'fontSize': 10, 'draggable': False, 'text-align':'center'}, 
                               className="three columns"),

                  dcc.Textarea(id='run-model-echo', readOnly='readonly', placeholder='',
                                value= str_metric,
                                rows='2',
                                style={'width': '98%', 'height':'50', 'background-color': '#F5F5F5', 
                                       'opacity': 0.5, 'fontSize': 25, 'draggable': False, 'text-align':'center', 'marginLeft':'15px'
                                       , 'marginRight':'15px'}, 
                               className="three columns"),                         


                  html.Hr(),
                  html.H4("Outputs for AUTO ARIMA MODEL", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading2"),


            dcc.Graph(id='line-graph1', style={'display': 'inline-block', 'width':'49%', 'marginLeft':'15px'},
            figure=go.Figure(data = [go.Scatter(
                                              x = df_pred['Date'],
                                              y = df_pred['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}),
                                   go.Scatter(
                                              x = df_pred['Date'],
                                              y = df_pred['Prediction'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),],
                                    layout=go.Layout(title={'text':'Close prices Prediction for AUTO ARIMA model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                                    ), className="Three columns"),
                            

          dcc.Textarea(id='run-model-echo1', readOnly='readonly', placeholder='',
                                value= str_msg1,
                                rows='30',
                                style={'display': 'inline', 'width': '48%', 'height':'1000', 'background-color': '#F5F5F5', 'marginLeft':'8px',
                                       'marginRight':'15px', 'opacity': 0.5, 'fontSize': 10, 'draggable': False, 'text-align':'center'}, 
                               className="three columns"),
                                     
          dcc.Textarea(id='run-model-echo', readOnly='readonly', placeholder='',
                                value= str_metric1,
                                rows='2',
                                style={'width': '98%', 'height':'50', 'background-color': '#F5F5F5', 
                                       'opacity': 0.5, 'fontSize': 25, 'draggable': False, 'text-align':'center', 'marginLeft':'15px',
                                       'marginRight':'15px'}, 
                               className="three columns"),

                  html.Hr(),
                  html.H4("Outputs for SARIMA MODEL", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading3"),


          dcc.Graph(id='line-graph2', style={'display': 'inline-block','width':'49%', 'marginLeft':'15px'},
          figure=go.Figure(data = [go.Scatter(
                                              x = df_pred['Date'],
                                              y = df_pred['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}),
                                   go.Scatter(
                                              x = df_pred['Date'],
                                              y = df_pred['Predicted_SARIMA'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),],
                                    layout=go.Layout(title={'text':'Close prices Prediction for SARIMA model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                                    ), className="six columns"),
                   
          
          dcc.Textarea(id='run-model-echo2', readOnly='readonly', placeholder='',
                                value= str_msg2,
                                rows='30',
                                style={'display': 'inline', 'width': '48%', 'height':'1000', 'background-color': '#F5F5F5', 'marginLeft':'8px',
                                       'marginRight':'15px', 'opacity': 0.5, 'fontSize': 10, 'draggable': False, 'text-align':'center'}, 
                               className="three columns"),

          dcc.Textarea(id='run-model-echo', readOnly='readonly', placeholder='',
                                value= str_metric2,
                                rows='2',
                                style={'width': '98%', 'height':'50', 'background-color': '#F5F5F5', 'marginRight':'15px',
                                       'opacity': '0.5', 'fontSize': 25, 'draggable': False, 'text-align':'center', 'marginLeft':'15px'}, 
                               className="three columns"),


          html.Hr(),
                  html.H4("Outputs for FB Prophet MODEL", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading4"),


          dcc.Graph(id='line-graph3', style={'display': 'inline-block', 'width':'48%', 'marginLeft':'15px', 'marginRight':'5px'},
          figure=go.Figure(data = [go.Scatter(
                                              x = df_pred['Date'],
                                              y = df_pred['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}),
                                   go.Scatter(
                                              x = df_past['ds'],
                                              y = df_past['yhat'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),
                                                      

                                   go.Scatter(
                                              x = df_past['ds'],
                                              y = df_past['yhat_lower'],
                                              mode = 'lines',
                                              fill = None,
                                              name = 'Lower range',
                                              line={'color': 'blue'}),
                                   go.Scatter(
                                              x = df_past['ds'],
                                              y = df_past['yhat_upper'],
                                              mode = 'lines',
                                              fill = 'tonexty',
                                              name = 'Upper range',
                                              line={'color': 'blue'}),
                    ],
                                    layout=go.Layout(title={'text':'Close prices Prediction for FB Prophet model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                           ),
                     className="six columns"),


            dcc.Graph(id='line-graph4', style={'display': 'inline-block', 'width':'48%', 'marginLeft':'10px'},
          figure=go.Figure(data = [go.Scatter(
                                              x = df_pred['Date'],
                                              y = df_pred['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}),
                                   go.Scatter(
                                              x = df_future['ds'],
                                              y = df_future['yhat'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),
                                   go.Scatter(
                                              x = df_future['ds'][len(df_future)-365:len(df_future)],
                                              y = df_future['yhat_lower'][len(df_future)-365:len(df_future)],
                                              mode = 'lines',
                                              fill = None,
                                              name = 'Lower range',
                                              line={'color': 'blue'}),
                                   go.Scatter(
                                              x = df_future['ds'][len(df_future)-365:len(df_future)],
                                              y = df_future['yhat_upper'][len(df_future)-365:len(df_future)],
                                              mode = 'lines',
                                              fill = 'tonexty',
                                              name = 'Upper range',
                                              line={'color': 'blue'}),],
                                    layout=go.Layout(title={'text':'Future Close prices Prediction for FB Prophet model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                          ),
                    className="six columns"),

          dcc.Textarea(id='run-model-echo', readOnly='readonly', placeholder='',
                                value= str_metric3,
                                rows='2',
                                style={'width': '98%', 'height':'50', 'background-color': '#F5F5F5', 'marginRight':'15px', 'marginLeft':'15px',
                                       'opacity': 0.5, 'fontSize': 25, 'draggable': False, 'text-align':'center'}, 
                               className="three columns")          
        ])

        return children
      else: 
        return dash.no_update

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



Coefficient of Determination for ARIMA Model 0.9345946265265235
Coefficient of Determination for AUTO ARIMA Model -1.161966156578258
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.50212D+00    |proj g|=  5.01889D-02

At iterate    5    f=  6.48879D+00    |proj g|=  3.62656D-03

At iterate   10    f=  6.48855D+00    |proj g|=  9.78749D-04

At iterate   15    f=  6.48846D+00    |proj g|=  1.82632D-03

At iterate   20    f=  6.48703D+00    |proj g|=  4.46817D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

 This problem is unconstrained.


Coefficient of Determination for SARIMA Model 0.9445271740681667
Initial log joint probability = -3.59645
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       365.747   0.000496574       74.6338           1           1      152   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     115       365.781   9.69171e-05       70.2269   1.254e-06       0.001      205  LS failed, Hessian reset 
     147        365.79   5.63319e-06       69.9835   8.973e-08       0.001      291  LS failed, Hessian reset 
     180        365.79   1.58612e-07       70.7968   2.333e-09       0.001      369  LS failed, Hessian reset 
     187        365.79    7.0224e-09       63.2274      0.2545      0.2545      379   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


...

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-FRI will be used.

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-FRI will be used.

 This problem is unconstrained.


At iterate   15    f=  5.87712D+00    |proj g|=  1.31238D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     16     21      1     0     0   2.846D-06   5.877D+00
  F =   5.8771223329122559     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
Coefficient of Determination for SARIMA Model 0.7667477333880579
Initial log joint probability = -10.2673
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        1655.3    0.00565826       112.861           1           1      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  #

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



Coefficient of Determination for ARIMA Model 0.8387597742818348
Coefficient of Determination for AUTO ARIMA Model 0.015834182632620153
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.69571D+00    |proj g|=  3.22261D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      3      6      1     0     0   2.838D-06   2.696D+00
  F =   2.6957058453793517     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/vamshi/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

 This problem is unconstrained.
